In [ ]:
!pip install Bio
!pip install pyfastx
!pip install transformers datasets evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 72.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 904.7/904.7 kB 16.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.6 

In [ ]:
import os
import pyfastx
import pandas as pd
import numpy as np
from collections import Counter

In [ ]:
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

Попробуем увеличить тренировочную выборку и отобрать последовательности так, чтобы экземпляры класса 1 и 0 не были похожи (не были частью одной и той же последовательности). Для генератора сделаем то же самое, но с 4 классами

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import csv
import transformers
from sklearn.model_selection import train_test_split

In [ ]:
classificatorPath = 'classificator_dataset.csv'
generatorPath = 'generator_dataset.csv'
n = 2000
classification_texts, classification_labels = [], []
generation_texts, generation_labels = [], []
ones, zeros = 0, 0
with open(classificatorPath) as csvFile:
  csvReader = csv.DictReader(csvFile, delimiter=';')
  for row in csvReader:
    if zeros == n and ones == n:
      break
    if int(row['label']) == 0 and zeros < n and ones == n:
      zeros += 1
    elif int(row['label']) == 1 and ones < n:
      ones += 1
    else:
      continue
    classification_texts.append(row['kmer'])
    classification_labels.append(int(row['label']))
t, a, c, g = 0, 0, 0, 0
with open(generatorPath) as csvFile:
  csvReader = csv.DictReader(csvFile, delimiter=';')
  for row in csvReader:
    if a == n and t == n and g == n and c == n:
      break
    if row['label'] == 'A' and a < n:
      a += 1
    elif row['label'] == 'C' and a == n and c < n:
      c += 1
    elif row['label'] == 'G' and c == n and g < n:
      g += 1
    elif row['label'] == 'T' and g == n and t < n:
      t += 1
    else:
      continue
    generation_texts.append(row['kmer'])
    generation_labels.append(row['label'])

In [ ]:
import random
def add_mismatches(texts):
  res = []
  nucleotides = ['A', 'T', 'C', 'G']
  for text in texts:
    mod_text = text
    i = random.randint(0, 8)
    mod_text = mod_text[:i] + nucleotides[random.randint(0, 3)] + mod_text[i+1:]
    i = random.randint(0, 8)
    mod_text = mod_text[:i] + nucleotides[random.randint(0, 3)] + mod_text[i+1:]
    res.append(mod_text)
  return res
test_texts, test_labels = add_mismatches(classification_texts), classification_labels

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(classification_texts, classification_labels, shuffle=True, test_size=.2)

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t6_8M_UR50D")
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)

In [ ]:
from transformers import EsmForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='results',          # output directory
    num_train_epochs=7,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=10,                # number of warmup steps for learning rate scheduler
    weight_decay=0.1,               # strength of weight decay
    logging_dir='logs',            # directory for storing logs
    logging_steps=10,
)

classificator = EsmForSequenceClassification.from_pretrained("facebook/esm2_t6_8M_UR50D")

trainer = Trainer(
    model=classificator,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()

Some weights of the model checkpoint at facebook/esm2_t6_8M_UR50D were not used when initializing EsmForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing EsmForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing EsmForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias', 'c

Step,Training Loss
10,0.693900
20,0.669900
30,0.645800
40,0.615000
50,0.594400
60,0.643100
70,0.548700
80,0.578300
90,0.605000
100,0.560200


TrainOutput(global_step=1400, training_loss=0.3971928655249732, metrics={'train_runtime': 441.75, 'train_samples_per_second': 50.707, 'train_steps_per_second': 3.169, 'total_flos': 11090783827200.0, 'train_loss': 0.3971928655249732, 'epoch': 7.0})

In [ ]:
from sklearn.metrics import classification_report
predicted_labels = [classificator(**tokenizer(kmer, return_tensors="pt")).logits.argmax().item() for kmer in test_texts]
target_names = ['non-conservative', 'conservative']
print(classification_report(test_labels, predicted_labels, target_names=target_names))

                  precision    recall  f1-score   support

non-conservative       0.68      0.75      0.71      2000
    conservative       0.72      0.65      0.68      2000

        accuracy                           0.70      4000
       macro avg       0.70      0.70      0.70      4000
    weighted avg       0.70      0.70      0.70      4000



Точность не изменилась. Precision, recall и f1-score для каждого класса примерно 0.7. Модель работает не идеально, но, по крайней мере, лучше, чем dumb model (предсказывает наугад, с точностью 0.5)

Точность генератора консервативных последовательностей определим по проценту сгенерированных последовательностей, которые классификатор определил как 1 (у обоих классов precision и recall совпадают, поэтому так можно сделать - False Positive и False Negative случаются с одинаковой вероятностью)

In [ ]:
id2label = {0: "A", 1: "C", 2: "G", 3: "T"}
label2id = {"A": 0, "C": 1, "G": 2, "T": 3}

In [ ]:
for i in range(len(generation_labels)):
  generation_labels[i] = label2id[generation_labels[i]]

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(generation_texts, generation_labels, test_size=.2)

In [ ]:
test_texts, test_labels = add_mismatches(generation_texts), generation_labels

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t6_8M_UR50D")
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)

In [ ]:
from transformers import EsmForSequenceClassification, AutoModelForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='results',          # output directory
    num_train_epochs=12,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=10,                # number of warmup steps for learning rate scheduler
    weight_decay=0.1,               # strength of weight decay
    logging_dir='logs',            # directory for storing logs
    logging_steps=10,
)

generator = EsmForSequenceClassification.from_pretrained("facebook/esm2_t6_8M_UR50D", num_labels=4, id2label=id2label, label2id=label2id)

trainer = Trainer(
    model=generator,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()

Some weights of the model checkpoint at facebook/esm2_t6_8M_UR50D were not used when initializing EsmForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing EsmForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing EsmForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias', 'c

Step,Training Loss
10,1.381300
20,1.402600
30,1.388800
40,1.389400
50,1.390300
60,1.388000
70,1.386600
80,1.389000
90,1.390000
100,1.387900


TrainOutput(global_step=4800, training_loss=1.217550923426946, metrics={'train_runtime': 1370.6429, 'train_samples_per_second': 56.032, 'train_steps_per_second': 3.502, 'total_flos': 34571635200000.0, 'train_loss': 1.217550923426946, 'epoch': 12.0})

In [ ]:
leng = 1000
generated_seq = 'ATGGGCCTCACCGT'
k = 9
ones = 0
for i in range(leng):
  nucl_id = generator(**tokenizer(generated_seq[i:i+k-1], return_tensors="pt")).logits.argmax().item()
  generated_seq += id2label[nucl_id]
  if classificator(**tokenizer(generated_seq[i:i+k], return_tensors="pt")).logits.argmax().item() == 1:
    ones += 1
ones/leng

0.86

86% сгенерированных последовательностей отнесены к классу 1, что является неплохим результатом